# Crystal Palace v. Arsenal - 2024-12-21 match analysis
*This notebook intends to walk through the theory and calculations I am currently using in my model to calculate probability of 1 x 2 result and a table of over / under goal projections. I will break the calculations out into utility functions stored in `../src/utils` for ease of use in future explorations. This is to document the calculations.*

### Instructions:

1. Ensure all cells in all notebooks in `/notebooks` directory have been run.
2. Define `home_side` and `away_side` in first cell. If making a comparison with odds from `03_retrieve_odds.ipynb` also enter matchday.
3. Click 'run all'
4. Probability for 1 x 2 and over / under can be viewed in the last cell
5. *Suggestion:* copy and paste notebook to `../output/< matchday_directory >/` and rename file. IMPORTANT: Update file path in lines 14 and 15 if needed if you make a copy.



### Import League Data & Define Sides

Define 'home_side' and 'away_side'
Ensure correct output



In [20]:
import pandas as pd
from scipy.stats import poisson
import numpy as np

"""
BUG: if copying from `notebooks/03_analysis_template_notebook.ipynb` to a directory in output
for storage, ensure the file path for reading the csv will need to be updated from '../data/processed/home_table.csv'
to '../../data/processed/home_table.csv' if frames are run again.

TLDR
if you are in ./notebooks/03_analysis_tempalte.csv you may run this cell as is. If you are not, check that following
read_csv commands read from the correct directory.
"""
home_dataframe = pd.read_csv('../data/processed/home_table.csv')
away_dataframe = pd.read_csv('../data/processed/away_table.csv')

"""
!!! INPUT TEAMS & MATCHDAY HERE !!!

wrapped in quotes, eg.

home_side = "Bournemouth"
away_side = "West Ham"
matchday = "2024-12-16" # (optional)

If there is a ValueError,
(1) check spelling against the error message. Must be exact.
(2) check capitalization
(3) check that it is wrapped in quotes

!!! INPUT TEAMS & MATCHDAY HERE !!!
"""
home_side = "Crystal Palace"
away_side = "Arsenal"
matchday = "2024-12-21" # (optional)

# validate input
clubs = set(home_dataframe['Team'].values)
if home_side not in clubs:
    raise ValueError(f"Invalid entry for 'home_side'. Expected one of {clubs}, but received '{home_side}'")
elif away_side not in clubs:
    raise ValueError(f"Invalid entry for 'away_side'. Expected one of {clubs}, but received '{away_side}'")
else:
    print(f"Ready to calculate probability for {home_side} v. {away_side}")

Ready to calculate probability for Crystal Palace v. Arsenal


### Calculate Projected Goals

In [21]:
# 2. grab their gpm_scored and gpm_conceded
home_side_gpm_scored = home_dataframe.loc[
    home_dataframe["Team"] == home_side,
    "gpm_scored"
].values[0]

home_side_gpm_conceded = home_dataframe.loc[
    home_dataframe["Team"] == home_side,
    "gpm_conceded"
].values[0]

print(home_side_gpm_scored, home_side_gpm_conceded)

away_side_gpm_scored = away_dataframe.loc[
    away_dataframe["Team"] == away_side,
    "gpm_scored"
].values[0]

away_side_gpm_conceded = away_dataframe.loc[
    away_dataframe["Team"] == away_side,
    "gpm_conceded"
].values[0]

print(away_side_gpm_scored, away_side_gpm_conceded)

# 3.
epl_home_avg_gpm_scored = round(sum(home_dataframe["gpm_scored"].values) / 20, 2)
epl_home_avg_gpm_conceded = round(sum(home_dataframe["gpm_conceded"].values) / 20, 2)
print(epl_home_avg_gpm_scored, epl_home_avg_gpm_conceded)

# 4. create a ratio (attack and defense rating) of their gpm to the league average gpm
home_attack_rating = home_side_gpm_scored / epl_home_avg_gpm_scored
away_defense_rating = away_side_gpm_conceded / epl_home_avg_gpm_scored
away_attack_rating = away_side_gpm_scored / epl_home_avg_gpm_conceded
home_defense_rating = home_side_gpm_conceded / epl_home_avg_gpm_conceded

print(home_attack_rating, away_defense_rating, away_attack_rating, home_defense_rating)

# home projected goals
home_projected_goals = home_attack_rating * away_defense_rating * epl_home_avg_gpm_scored
# away projected goals
away_projected_goals = away_attack_rating * home_defense_rating * epl_home_avg_gpm_conceded
print("\n ================== projected_goals ==================== \n")
print(f"HOME {home_side} projected goals:", home_projected_goals)
print(f"AWAY {away_side} projected goals: ", away_projected_goals)

0.75 1.25
1.5 1.125
1.61 1.4
0.4658385093167702 0.6987577639751552 1.0714285714285714 0.8928571428571429

 ================== projected_goals ==================== 

HOME Crystal Palace projected goals: 0.5240683229813664
AWAY Arsenal projected goals:  1.3392857142857142


### Poisson Distribution of Goal Probabilities

In [22]:
# Poisson probability for home team goals 0 - 8
home_score_prob = []
away_score_prob = []
for i in range(9):
    prob = poisson.pmf(i, home_projected_goals)
    home_score_prob.append(prob)
for i in range(9):
    prob = poisson.pmf(i, away_projected_goals)
    away_score_prob.append(prob)

home_score_prob = np.array(home_score_prob)
away_score_prob = np.array(away_score_prob)
# print(home_score_prob)
# print(away_score_prob)

exact_score_prob = np.outer(away_score_prob, home_score_prob)
print(f"{home_side} v. {away_side}", exact_score_prob)

Crystal Palace v. Arsenal [[1.55151373e-01 8.13099200e-02 2.13059767e-02 3.72192916e-03
  4.87636293e-04 5.11109469e-05 4.46427137e-06 3.34226173e-07
  2.18946687e-08]
 [2.07792018e-01 1.08897214e-01 2.85347902e-02 4.98472655e-03
  6.53084321e-04 6.84521610e-05 5.97893487e-06 4.47624339e-07
  2.93232171e-08]
 [1.39146440e-01 7.29222417e-02 1.91081185e-02 3.33798653e-03
  4.37333251e-04 4.58385007e-05 4.00375103e-06 2.99748441e-07
  1.96360829e-08]
 [6.21189466e-02 3.25545722e-02 8.53041002e-03 1.49017256e-03
  1.95238058e-04 2.04636164e-05 1.78738885e-06 1.33816268e-07
  8.76610842e-09]
 [2.07987544e-02 1.08999684e-02 2.85616407e-03 4.98941705e-04
  6.53698856e-05 6.85165727e-06 5.98456089e-07 4.48045541e-08
  2.93508094e-09]
 [5.57109494e-03 2.91963438e-03 7.65043947e-04 1.33645100e-04
  1.75097908e-05 1.83526534e-06 1.60300738e-07 1.20012199e-08
  7.86182395e-10]
 [1.24354798e-03 6.51704103e-04 1.70768738e-04 2.98314954e-05
  3.90843545e-06 4.09657442e-07 3.57814148e-08 2.67884372e-0

### Under Market

In [23]:
thresholds = [0.5, 1.5, 2.5, 3.5]
sums = []

for t in thresholds:
    mask = np.add.outer(
        np.arange(
            exact_score_prob.shape[0]
        ),
        np.arange(
            exact_score_prob.shape[1]
        )
    ) <= t
    # print(mask)
    sums.append(exact_score_prob[mask].sum())

# print(sums)
under_market_df = pd.DataFrame({
    "Goals": thresholds,
    "Prob": sums
})
under_market_df["Implied Odds"] = 1 / under_market_df["Prob"]
print("~  ~ under market ~  ~\n", f"{home_side} v. {away_side}\n", under_market_df)

~  ~ under market ~  ~
 Crystal Palace v. Arsenal
    Goals      Prob  Implied Odds
0    0.5  0.155151      6.445318
1    1.5  0.444253      2.250968
2    2.5  0.713603      1.401340
3    3.5  0.880901      1.135202


In [24]:
thresholds = [0.5, 1.5, 2.5, 3.5]
sums = []

for t in thresholds:
    mask = np.add.outer(
        np.arange(
            exact_score_prob.shape[0]
        ),
        np.arange(
            exact_score_prob.shape[1]
        )
    ) >= t
    sums.append(exact_score_prob[mask].sum())

# print(sums)
over_market_df = pd.DataFrame({
    "Goals": thresholds,
    "Prob": sums
})

over_market_df["Implied Odds"] = 1 / over_market_df["Prob"]
print("~  ~ over market ~  ~\n", f"{home_side} v. {away_side}\n", over_market_df)

~  ~ over market ~  ~
 Crystal Palace v. Arsenal
    Goals      Prob  Implied Odds
0    0.5  0.844837      1.183660
1    1.5  0.555735      1.799418
2    2.5  0.286386      3.491797
3    3.5  0.119088      8.397179


### 1 x 2 Probability

In [25]:
# Poisson probability for home team goals 0 - 8
home_score_prob = []
away_score_prob = []
for i in range(9):
    prob = poisson.pmf(i, home_projected_goals)
    home_score_prob.append(prob)
for i in range(9):
    prob = poisson.pmf(i, away_projected_goals)
    away_score_prob.append(prob)

home_score_prob = np.array(home_score_prob)
away_score_prob = np.array(away_score_prob)
# print(home_score_prob)
# print(away_score_prob)

exact_score_prob = np.outer(away_score_prob, home_score_prob)

# print(exact_score_prob)
home_win_prob = np.sum(np.triu(exact_score_prob, k=1))
away_win_prob = np.sum(np.tril(exact_score_prob, k=-1))
draw_prob = np.trace(exact_score_prob)

# prepare data in readable dataframe
data = {
    "winner": [f"{home_side} (1)", "Draw (x)", f"{away_side} (2)"],
    "prob": [home_win_prob, draw_prob, away_win_prob],
}
result_df = pd.DataFrame(data)
result_df['implied_odds'] = 1 / result_df['prob']
print("      ~  ~ moneyline market ~  ~\n", result_df)

      ~  ~ moneyline market ~  ~
                winner      prob  implied_odds
0  Crystal Palace (1)  0.145180      6.888016
1            Draw (x)  0.284714      3.512295
2         Arsenal (2)  0.570095      1.754095


# Full Results Summary:

In [26]:
# helper function to translate discrepency between Understat and Odds-Api naming
def translate(string):

    translations = {
        "Brighton": "Brighton and Hove Albion",
        "Ipswich": "Ipswich Town",
        "Leicester": "Leicester City",
        "Tottenham": "Tottenham Hotspur",
        "West Ham": "West Ham United"
    }

    if string in translations:
        return translations[string]
    return string

home_side, away_side = translate(home_side), translate(away_side)

match_string = f"{home_side} v. {away_side}"

print(f"CALCULATED PROBABILITIES FOR {home_side} v. {away_side}")
print("\n\n        ~ ~ moneyline market ~ ~\n\n", result_df)
print("\n\n          ~ ~ over market ~ ~ \n\n", over_market_df)
print("\n\n          ~ ~ under market ~ ~ \n\n", under_market_df)

try:
    odds_df = pd.read_csv(f"../data/processed/odds/odds_matchday_{matchday}.csv")
    filtered_odds = odds_df.loc[odds_df["match"] == match_string]
    implied = {
        "1": 1 / filtered_odds["1"],
        "x": 1 / filtered_odds["x"],
        "2": 1 / filtered_odds["2"],
        "point": filtered_odds["ovr_und_point"],
        "over": 1 / filtered_odds["over"],
        "under": 1 / filtered_odds["under"]
    }
    implied_odds_df = pd.DataFrame(implied)

    print(f"\n ~~ MyBookie Odds for {match_string} ~~\n",filtered_odds.drop(columns=["match"]))
    print(f"\n ~~ MyBookie Odds IMPLIED for {match_string} ~~\n", implied_odds_df)
except:
    print(f"No data for sportsbook odds on matchday {matchday}. To view against real odds, run `notebooks/03_retrieve_odds.ipynb` ran correctly and ensure matchday is correct in the first cell of this notebook")


CALCULATED PROBABILITIES FOR Crystal Palace v. Arsenal


        ~ ~ moneyline market ~ ~

                winner      prob  implied_odds
0  Crystal Palace (1)  0.145180      6.888016
1            Draw (x)  0.284714      3.512295
2         Arsenal (2)  0.570095      1.754095


          ~ ~ over market ~ ~ 

    Goals      Prob  Implied Odds
0    0.5  0.844837      1.183660
1    1.5  0.555735      1.799418
2    2.5  0.286386      3.491797
3    3.5  0.119088      8.397179


          ~ ~ under market ~ ~ 

    Goals      Prob  Implied Odds
0    0.5  0.155151      6.445318
1    1.5  0.444253      2.250968
2    2.5  0.713603      1.401340
3    3.5  0.880901      1.135202

 ~~ MyBookie Odds for Crystal Palace v. Arsenal ~~
      1    x     2  ovr_und_point  over  under
4  5.7  4.0  1.55            2.5  1.83    1.9

 ~~ MyBookie Odds IMPLIED for Crystal Palace v. Arsenal ~~
           1     x         2  point      over     under
4  0.175439  0.25  0.645161    2.5  0.546448  0.526316


# Conclusion and Pick
I'm scared of the Under pick. That's the value that's called out here. I'm leaving it, but my eyes on it.
### My Pick:

**NOBET** 